In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os


In [2]:
INIT_LR = 1e-4
EPOCHS = 20
#20
BS = 32

In [3]:
import glob,os

path = r'C:\Users\vutukuri.venkatesh\Desktop\stream_training\material\face-mask-detector\face-mask-detector\dataset\with_mask'
files = [f for f in glob.glob(path + "*/*.JPG", recursive=True)]

path = r'C:\Users\vutukuri.venkatesh\Desktop\stream_training\material\face-mask-detector\face-mask-detector\dataset\without_mask'
files1 = [f for f in glob.glob(path + "*/*.JPG", recursive=True)]
#print(files1)

In [4]:
data = []
labels = []

In [5]:
for imagePath in files:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)
	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

for imagePath in files1:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)
	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

In [6]:
data[0]

array([[[-0.5529412 , -0.5058824 , -0.7254902 ],
        [-0.6392157 , -0.5686275 , -0.78039217],
        [-0.654902  , -0.5764706 , -0.7647059 ],
        ...,
        [-0.6862745 , -0.654902  , -0.7882353 ],
        [-0.7647059 , -0.7176471 , -0.92156863],
        [-0.7176471 , -0.6784314 , -0.85882354]],

       [[-0.6       , -0.5529412 , -0.75686276],
        [-0.64705884, -0.58431375, -0.77254903],
        [-0.6392157 , -0.56078434, -0.7490196 ],
        ...,
        [-0.49019605, -0.4588235 , -0.5764706 ],
        [-0.81960785, -0.75686276, -0.94509804],
        [-0.70980394, -0.64705884, -0.8117647 ]],

       [[-0.62352943, -0.5764706 , -0.78039217],
        [-0.6392157 , -0.5764706 , -0.7647059 ],
        [-0.62352943, -0.54509807, -0.73333335],
        ...,
        [-0.6       , -0.5529412 , -0.6627451 ],
        [-0.827451  , -0.7647059 , -0.92941177],
        [-0.7019608 , -0.6313726 , -0.78039217]],

       ...,

       [[-0.54509807, -0.42745095, -0.73333335],
        [-0

In [7]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [8]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [9]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [10]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

In [11]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [12]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [13]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [14]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [15]:
for layer in baseModel.layers:
	layer.trainable = False

In [16]:
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

[INFO] compiling model...


In [17]:
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

Epoch 1/20
34/34 [==============================] - 38s 1s/step - loss: 0.4566 - accuracy: 0.7753 - val_loss: 0.1359 - val_accuracy: 0.9710
Epoch 2/20
34/34 [==============================] - 37s 1s/step - loss: 0.1387 - accuracy: 0.9635 - val_loss: 0.0667 - val_accuracy: 0.9855
Epoch 3/20
34/34 [==============================] - 37s 1s/step - loss: 0.0750 - accuracy: 0.9850 - val_loss: 0.0467 - val_accuracy: 0.9855
Epoch 4/20
34/34 [==============================] - 35s 1s/step - loss: 0.0523 - accuracy: 0.9860 - val_loss: 0.0365 - val_accuracy: 0.9928
Epoch 5/20
34/34 [==============================] - 35s 1s/step - loss: 0.0385 - accuracy: 0.9925 - val_loss: 0.0317 - val_accuracy: 0.9928
Epoch 6/20
34/34 [==============================] - 33s 984ms/step - loss: 0.0410 - accuracy: 0.9869 - val_loss: 0.0253 - val_accuracy: 0.9964
Epoch 7/20
34/34 [==============================] - 34s 997ms/step - loss: 0.0343 - accuracy: 0.9897 - val_loss: 0.0222 - val_accuracy: 0.9964
Epoch 8/20
34/

In [18]:
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

   with_mask       1.00      1.00      1.00       138
without_mask       1.00      1.00      1.00       138

    accuracy                           1.00       276
   macro avg       1.00      1.00      1.00       276
weighted avg       1.00      1.00      1.00       276



In [19]:
model.save("final_model1.pkl")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: final_model1.pkl\assets


## For Video Streaming

In [ ]:
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# from tensorflow.keras.preprocessing.image import img_to_array
# from imutils.video import VideoStream
# import numpy as np
# import imutils
# import time
# import cv2
# import os

In [ ]:
# def detect_and_predict_mask(frame, faceNet, maskNet):
# 	# grab the dimensions of the frame and then construct a blob
# 	# from it
# 	(h, w) = frame.shape[:2]
# 	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
# 		(104.0, 177.0, 123.0))

# 	# pass the blob through the network and obtain the face detections
# 	faceNet.setInput(blob)
# 	detections = faceNet.forward()

# 	# initialize our list of faces, their corresponding locations,
# 	# and the list of predictions from our face mask network
# 	faces = []
# 	locs = []
# 	preds = []

# 	# loop over the detections
# 	for i in range(0, detections.shape[2]):
# 		# extract the confidence (i.e., probability) associated with
# 		# the detection
# 		confidence = detections[0, 0, i, 2]

# 		# filter out weak detections by ensuring the confidence is
# 		# greater than the minimum confidence
# 		if confidence > 0.5:
# 			# compute the (x, y)-coordinates of the bounding box for
# 			# the object
# 			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
# 			(startX, startY, endX, endY) = box.astype("int")

# 			# ensure the bounding boxes fall within the dimensions of
# 			# the frame
# 			(startX, startY) = (max(0, startX), max(0, startY))
# 			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

# 			# extract the face ROI, convert it from BGR to RGB channel
# 			# ordering, resize it to 224x224, and preprocess it
# 			face = frame[startY:endY, startX:endX]
# 			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
# 			face = cv2.resize(face, (224, 224))
# 			face = img_to_array(face)
# 			face = preprocess_input(face)

# 			# add the face and bounding boxes to their respective
# 			# lists
# 			faces.append(face)
# 			locs.append((startX, startY, endX, endY))

# 	# only make a predictions if at least one face was detected
# 	if len(faces) > 0:
# 		# for faster inference we'll make batch predictions on *all*
# 		# faces at the same time rather than one-by-one predictions
# 		# in the above `for` loop
# 		faces = np.array(faces, dtype="float32")
# 		preds = maskNet.predict(faces, batch_size=32)

# 	# return a 2-tuple of the face locations and their corresponding
# 	# locations
# 	return (locs, preds)

In [ ]:
# prototxtPath =  r'C:\Users\vutukuri.venkatesh\Desktop\stream_training\material\face-mask-detector\face-mask-detector\face_detector\deploy.prototxt.'
# weightsPath = r'C:\Users\vutukuri.venkatesh\Desktop\stream_training\material\face-mask-detector\face-mask-detector\face_detector\res10_300x300_ssd_iter_140000.caffemodel'
# faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [ ]:
# print("[INFO] starting video stream...")
# vs = VideoStream(src=0).start()
# time.sleep(2.0)
# while True:
# 	# grab the frame from the threaded video stream and resize it
# 	# to have a maximum width of 400 pixels
# 	frame = vs.read()
# 	frame = imutils.resize(frame, width=400)
# 	maskNet=model
# 	# face mask or not
# 	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

# 	# loop over the detected face locations and their corresponding
# 	# locations
# 	for (box, pred) in zip(locs, preds):
# 		# unpack the bounding box and predictions
# 		(startX, startY, endX, endY) = box
# 		(mask, withoutMask) = pred

# 		# determine the class label and color we'll use to draw
# 		# the bounding box and text
# 		label = "Mask" if mask > withoutMask else "No Mask"
# 		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

# 		# include the probability in the label
# 		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

# 		# display the label and bounding box rectangle on the output
# 		# frame
# 		cv2.putText(frame, label, (startX, startY - 10),
# 			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
# 		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

# 	# show the output frame
# 	cv2.imshow("Frame", frame)
# 	key = cv2.waitKey(1) & 0xFF

# 	# if the `q` key was pressed, break from the loop
# 	if key == ord("q"):
# 		vs.stream.release()
# 		break

# # do a bit of cleanup
# cv2.destroyAllWindows()
# vs.stop()

## checking for a image

In [ ]:
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# from tensorflow.keras.preprocessing.image import img_to_array
# import numpy as np
# import cv2
# import os

# print("[INFO] loading face detector model...")
# prototxtPath =  r'C:\Users\vutukuri.venkatesh\Desktop\stream_training\material\face-mask-detector\face-mask-detector\face_detector\deploy.prototxt.'
# weightsPath = r'C:\Users\vutukuri.venkatesh\Desktop\stream_training\material\face-mask-detector\face-mask-detector\face_detector\res10_300x300_ssd_iter_140000.caffemodel'
# net = cv2.dnn.readNet(prototxtPath, weightsPath)


In [ ]:
# file3 = r'C:\Users\vutukuri.venkatesh\Desktop\stream_training\material\face-mask-detector\face-mask-detector\examples\example_01.PNG'
# image = cv2.imread(file3)
# orig = image.copy()
# (h, w) = image.shape[:2]
# # construct a blob from the image
# blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),(104.0, 177.0, 123.0))
# # pass the blob through the network and obtain the face detections
# print("[INFO] computing face detections...")
# net.setInput(blob)
# detections = net.forward()

In [ ]:
# for i in range(0, detections.shape[2]):
# 	# extract the confidence (i.e., probability) associated with
# 	# the detection
# 	confidence = detections[0, 0, i, 2]
# 	# filter out weak detections by ensuring the confidence is
# 	# greater than the minimum confidence
# 	if confidence > 0.5:
# 		# compute the (x, y)-coordinates of the bounding box for
# 		# the object
# 		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
# 		(startX, startY, endX, endY) = box.astype("int")
# 		# ensure the bounding boxes fall within the dimensions of
# 		# the frame
# 		(startX, startY) = (max(0, startX), max(0, startY))
# 		(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

In [ ]:
# 		face = image[startY:endY, startX:endX]
# 		face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
# 		face = cv2.resize(face, (224, 224))
# 		face = img_to_array(face)
# 		face = preprocess_input(face)
# 		face = np.expand_dims(face, axis=0)
# 		# pass the face through the model to determine if the face
# 		# has a mask or not
# 		(mask, withoutMask) = model.predict(face)[0]

In [ ]:
# 		label = "Mask" if mask > withoutMask else "No Mask"
# 		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
# 		# include the probability in the label
# 		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
# 		# display the label and bounding box rectangle on the output
# 		# frame
# 		cv2.putText(image, label, (startX, startY - 10),
# 			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
# 		cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
# # show the output image
# cv2.imshow("Output", image)
# cv2.waitKey(0)